In [47]:
import pandas as pd
import numpy as np

from urllib.parse import quote    
from urllib.request import urlopen

import sys
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from sqlalchemy import create_engine

#import pymysql.cursors
#pymysql.install_as_MySQLdb()
#import MySQLdb
from pandas.io.json import json_normalize
import time
#from pyproj import Proj, transform
#import folium
#from folium.plugins import MarkerCluster, MiniMap
import re

# subway station

In [78]:
# 지하철 데이터 
subway_coord = pd.read_csv('seoul_subway_station_information.csv')

In [77]:
subway_coord.to_csv('seoul_subway_station_information.csv',index=False)

# subway monthly timelog

In [7]:
api_key = '4c6e727a6c6779653532546b576f6e'
base_url = 'http://openapi.seoul.go.kr:8088/{}/json/CardSubwayTime/{}/{}/{}/'

year_range = ['2015','2016','2017','2018','2019']
month_range = ['01','02','03','04','05','06','07','08','09','10','11','12']

for j in year_range:
    start_time = time.time()
    table_name = 'subway_monthly_timelog_'+j
    monthly_df = pd.DataFrame()
    
    for i in month_range:
        ym_value = j + i
        
        num = 1
        while True:
            try:

                req = requests.get(base_url.format(api_key,str(num),str(num+999),ym_value), headers = {'user-agent': \
                                                ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36')})

                subway_json = req.json()
                sample_df = json_normalize(subway_json['CardSubwayTime']['row'])
                
                monthly_df = monthly_df.append(sample_df)

                #sample_df.to_sql(name=table_name,con=engine,if_exists='append')


                print('지하철 데이터 {} ~ {} row 수집중'.format(str(num),str(num+999)))
                num +=1000
            except:
                break
        print('--------------------'+j+'년'+i+'월 데이터 수집 완료---------------------')
        
        
    print('============================================================================')
    print('=============================='+j+' 연도 테이블 생성 완료========================')
    print('============================================================================')
    print('연평균 수집 소요 시간(초) : ', time.time()-start_time)
    monthly_df.index = range(len(monthly_df))

    monthly_df.to_csv(table_name+'.csv')



/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년01월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년02월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년03월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년04월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년05월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년06월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년07월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년08월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년09월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년10월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년11월 데이터 수집 완료---------------------
지하철 데이터 1 ~ 1000 row 수집중
--------------------2015년12월 데이터 수집 완료--

In [13]:
subway_monthlylog2015 = pd.read_csv('subway_monthly_timelog_2015.csv')
subway_monthlylog2016 = pd.read_csv('subway_monthly_timelog_2016.csv')
subway_monthlylog2017 = pd.read_csv('subway_monthly_timelog_2017.csv')
subway_monthlylog2018 = pd.read_csv('subway_monthly_timelog_2018.csv')
subway_monthlylog2019 = pd.read_csv('subway_monthly_timelog_2019.csv')

In [67]:
subway_monthlylog2015.drop(columns='Unnamed: 0',inplace=True)
subway_monthlylog2016.drop(columns='Unnamed: 0',inplace=True)
subway_monthlylog2017.drop(columns='Unnamed: 0',inplace=True)
subway_monthlylog2018.drop(columns='Unnamed: 0',inplace=True)
subway_monthlylog2019.drop(columns='Unnamed: 0',inplace=True)

In [68]:
subway_monthly = pd.concat([subway_monthlylog2015,subway_monthlylog2016,subway_monthlylog2017,\
                           subway_monthlylog2018,subway_monthlylog2019])

In [82]:
subway_monthly.to_csv('seoul_subway_monthly_timely_boarding_2015_2019.csv',index=False)

# subway dailylog

In [8]:
api_key = '7a4d4a55776779653838644e786969'
base_url = 'http://openapi.seoul.go.kr:8088/{}/json/CardSubwayStatsNew/{}/{}/{}'

year_range = ['2015','2016','2017','2018','2019']
date_range = pd.date_range(start='20150101', end='20191231').strftime('%Y%m%d').tolist()
month_range = []
for i in date_range:
    month_range.append(i[4:])
day_range = pd.Series(month_range).unique().tolist()[:-1]   

In [10]:
for i in year_range:
    start_time = time.time()
    db_name = 'subway_dailylog_' + i
    
    dailylog_df = pd.DataFrame()
    for j in day_range:
        ym = i+j
        
        num = 1
        
        while True:
            try:
                req = requests.get(base_url.format(api_key,str(num),str(num+999),ym), headers = {'user-agent': \
                                                        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36')})

                json_data = req.json()
                sample_df = json_normalize(json_data['CardSubwayStatsNew']['row'])
                
                dailylog_df = dailylog_df.append(sample_df)
                #sample_df.to_sql(name=db_name,con=engine,if_exists='append')
                
                #print('지하철 데이터 {} ~ {} row 수집중'.format(str(num),str(num+999)))
                print(i+j+' 데이터 적재 완료')
                num +=1000
            except:
                break
                
    print('============================================================================')
    print('=============================='+i+' 연도 테이블 생성 완료========================')
    print('============================================================================')
    print('연평균 수집 소요 시간(초) : ', time.time()-start_time)
    
    dailylog_df.index = range(len(dailylog_df))
    dailylog_df.to_csv(db_name+'.csv')

        

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


20150101 데이터 적재 완료
20150102 데이터 적재 완료
20150103 데이터 적재 완료
20150104 데이터 적재 완료
20150105 데이터 적재 완료
20150106 데이터 적재 완료
20150107 데이터 적재 완료
20150108 데이터 적재 완료
20150109 데이터 적재 완료
20150110 데이터 적재 완료
20150111 데이터 적재 완료
20150112 데이터 적재 완료
20150113 데이터 적재 완료
20150114 데이터 적재 완료
20150115 데이터 적재 완료
20150116 데이터 적재 완료
20150117 데이터 적재 완료
20150118 데이터 적재 완료
20150119 데이터 적재 완료
20150120 데이터 적재 완료
20150121 데이터 적재 완료
20150122 데이터 적재 완료
20150123 데이터 적재 완료
20150124 데이터 적재 완료
20150125 데이터 적재 완료
20150126 데이터 적재 완료
20150127 데이터 적재 완료
20150128 데이터 적재 완료
20150129 데이터 적재 완료
20150130 데이터 적재 완료
20150131 데이터 적재 완료
20150201 데이터 적재 완료
20150202 데이터 적재 완료
20150203 데이터 적재 완료
20150204 데이터 적재 완료
20150205 데이터 적재 완료
20150206 데이터 적재 완료
20150207 데이터 적재 완료
20150208 데이터 적재 완료
20150209 데이터 적재 완료
20150210 데이터 적재 완료
20150211 데이터 적재 완료
20150212 데이터 적재 완료
20150213 데이터 적재 완료
20150214 데이터 적재 완료
20150215 데이터 적재 완료
20150216 데이터 적재 완료
20150217 데이터 적재 완료
20150218 데이터 적재 완료
20150219 데이터 적재 완료
20150220 데이터 적재 완료
20150221 데이터 적재 완료
20150222 데이터

20160224 데이터 적재 완료
20160225 데이터 적재 완료
20160226 데이터 적재 완료
20160227 데이터 적재 완료
20160228 데이터 적재 완료
20160301 데이터 적재 완료
20160302 데이터 적재 완료
20160303 데이터 적재 완료
20160304 데이터 적재 완료
20160305 데이터 적재 완료
20160306 데이터 적재 완료
20160307 데이터 적재 완료
20160308 데이터 적재 완료
20160309 데이터 적재 완료
20160310 데이터 적재 완료
20160311 데이터 적재 완료
20160312 데이터 적재 완료
20160313 데이터 적재 완료
20160314 데이터 적재 완료
20160315 데이터 적재 완료
20160316 데이터 적재 완료
20160317 데이터 적재 완료
20160318 데이터 적재 완료
20160319 데이터 적재 완료
20160320 데이터 적재 완료
20160321 데이터 적재 완료
20160322 데이터 적재 완료
20160323 데이터 적재 완료
20160324 데이터 적재 완료
20160325 데이터 적재 완료
20160326 데이터 적재 완료
20160327 데이터 적재 완료
20160328 데이터 적재 완료
20160329 데이터 적재 완료
20160330 데이터 적재 완료
20160331 데이터 적재 완료
20160401 데이터 적재 완료
20160402 데이터 적재 완료
20160403 데이터 적재 완료
20160404 데이터 적재 완료
20160405 데이터 적재 완료
20160406 데이터 적재 완료
20160407 데이터 적재 완료
20160408 데이터 적재 완료
20160409 데이터 적재 완료
20160410 데이터 적재 완료
20160411 데이터 적재 완료
20160412 데이터 적재 완료
20160413 데이터 적재 완료
20160414 데이터 적재 완료
20160415 데이터 적재 완료
20160416 데이터 적재 완료
20160417 데이터

20170418 데이터 적재 완료
20170419 데이터 적재 완료
20170420 데이터 적재 완료
20170421 데이터 적재 완료
20170422 데이터 적재 완료
20170423 데이터 적재 완료
20170424 데이터 적재 완료
20170425 데이터 적재 완료
20170426 데이터 적재 완료
20170427 데이터 적재 완료
20170428 데이터 적재 완료
20170429 데이터 적재 완료
20170430 데이터 적재 완료
20170501 데이터 적재 완료
20170502 데이터 적재 완료
20170503 데이터 적재 완료
20170504 데이터 적재 완료
20170505 데이터 적재 완료
20170506 데이터 적재 완료
20170507 데이터 적재 완료
20170508 데이터 적재 완료
20170509 데이터 적재 완료
20170510 데이터 적재 완료
20170511 데이터 적재 완료
20170512 데이터 적재 완료
20170513 데이터 적재 완료
20170514 데이터 적재 완료
20170515 데이터 적재 완료
20170516 데이터 적재 완료
20170517 데이터 적재 완료
20170518 데이터 적재 완료
20170519 데이터 적재 완료
20170520 데이터 적재 완료
20170521 데이터 적재 완료
20170522 데이터 적재 완료
20170523 데이터 적재 완료
20170524 데이터 적재 완료
20170525 데이터 적재 완료
20170526 데이터 적재 완료
20170527 데이터 적재 완료
20170528 데이터 적재 완료
20170529 데이터 적재 완료
20170530 데이터 적재 완료
20170531 데이터 적재 완료
20170601 데이터 적재 완료
20170602 데이터 적재 완료
20170603 데이터 적재 완료
20170604 데이터 적재 완료
20170605 데이터 적재 완료
20170606 데이터 적재 완료
20170607 데이터 적재 완료
20170608 데이터 적재 완료
20170609 데이터

20180610 데이터 적재 완료
20180611 데이터 적재 완료
20180612 데이터 적재 완료
20180613 데이터 적재 완료
20180614 데이터 적재 완료
20180615 데이터 적재 완료
20180616 데이터 적재 완료
20180617 데이터 적재 완료
20180618 데이터 적재 완료
20180619 데이터 적재 완료
20180620 데이터 적재 완료
20180621 데이터 적재 완료
20180622 데이터 적재 완료
20180623 데이터 적재 완료
20180624 데이터 적재 완료
20180625 데이터 적재 완료
20180626 데이터 적재 완료
20180627 데이터 적재 완료
20180628 데이터 적재 완료
20180629 데이터 적재 완료
20180630 데이터 적재 완료
20180701 데이터 적재 완료
20180702 데이터 적재 완료
20180703 데이터 적재 완료
20180704 데이터 적재 완료
20180705 데이터 적재 완료
20180706 데이터 적재 완료
20180707 데이터 적재 완료
20180708 데이터 적재 완료
20180709 데이터 적재 완료
20180710 데이터 적재 완료
20180711 데이터 적재 완료
20180712 데이터 적재 완료
20180713 데이터 적재 완료
20180714 데이터 적재 완료
20180715 데이터 적재 완료
20180716 데이터 적재 완료
20180717 데이터 적재 완료
20180718 데이터 적재 완료
20180719 데이터 적재 완료
20180720 데이터 적재 완료
20180721 데이터 적재 완료
20180722 데이터 적재 완료
20180723 데이터 적재 완료
20180724 데이터 적재 완료
20180725 데이터 적재 완료
20180726 데이터 적재 완료
20180727 데이터 적재 완료
20180728 데이터 적재 완료
20180729 데이터 적재 완료
20180730 데이터 적재 완료
20180731 데이터 적재 완료
20180801 데이터

20190802 데이터 적재 완료
20190803 데이터 적재 완료
20190804 데이터 적재 완료
20190805 데이터 적재 완료
20190806 데이터 적재 완료
20190807 데이터 적재 완료
20190808 데이터 적재 완료
20190809 데이터 적재 완료
20190810 데이터 적재 완료
20190811 데이터 적재 완료
20190812 데이터 적재 완료
20190813 데이터 적재 완료
20190814 데이터 적재 완료
20190815 데이터 적재 완료
20190816 데이터 적재 완료
20190817 데이터 적재 완료
20190818 데이터 적재 완료
20190819 데이터 적재 완료
20190820 데이터 적재 완료
20190821 데이터 적재 완료
20190822 데이터 적재 완료
20190823 데이터 적재 완료
20190824 데이터 적재 완료
20190825 데이터 적재 완료
20190826 데이터 적재 완료
20190827 데이터 적재 완료
20190828 데이터 적재 완료
20190829 데이터 적재 완료
20190830 데이터 적재 완료
20190831 데이터 적재 완료
20190901 데이터 적재 완료
20190902 데이터 적재 완료
20190903 데이터 적재 완료
20190904 데이터 적재 완료
20190905 데이터 적재 완료
20190906 데이터 적재 완료
20190907 데이터 적재 완료
20190908 데이터 적재 완료
20190909 데이터 적재 완료
20190910 데이터 적재 완료
20190911 데이터 적재 완료
20190912 데이터 적재 완료
20190913 데이터 적재 완료
20190914 데이터 적재 완료
20190915 데이터 적재 완료
20190916 데이터 적재 완료
20190917 데이터 적재 완료
20190918 데이터 적재 완료
20190919 데이터 적재 완료
20190920 데이터 적재 완료
20190921 데이터 적재 완료
20190922 데이터 적재 완료
20190923 데이터

In [12]:
subway_dailylog2015 = pd.read_csv('subway_dailylog_2015.csv')
subway_dailylog2016 = pd.read_csv('subway_dailylog_2016.csv')
subway_dailylog2017 = pd.read_csv('subway_dailylog_2017.csv')
subway_dailylog2018 = pd.read_csv('subway_dailylog_2018.csv')
subway_dailylog2019 = pd.read_csv('subway_dailylog_2019.csv')

In [62]:
subway_dailylog2015.drop(columns='Unnamed: 0',inplace=True)
subway_dailylog2016.drop(columns='Unnamed: 0',inplace=True)
subway_dailylog2017.drop(columns='Unnamed: 0',inplace=True)
subway_dailylog2018.drop(columns='Unnamed: 0',inplace=True)
subway_dailylog2019.drop(columns='Unnamed: 0',inplace=True)

In [63]:
subway_dailylog = pd.concat([subway_dailylog2015,subway_dailylog2016,subway_dailylog2017,\
                            subway_dailylog2018,subway_dailylog2019])

In [84]:
subway_dailylog.to_csv('seoul_subway_daily_boarding_2015_2019.csv',index=False)

# subway daily timelog

In [11]:
#2014
subway_timelog0 = pd.read_csv('./일별시간별지하철승하차/서울교통공사 2014년 일별 역별 시간대별 승하차인원(1_8호선).csv',encoding='euc-kr')
#2015
subway_timelog1 = pd.read_csv('./일별시간별지하철승하차/서울교통공사 2015년 일별 역별 시간대별 승하차인원(1_8호선).csv',encoding='euc-kr')
#2016
subway_timelog2 = pd.read_csv('./일별시간별지하철승하차/서울교통공사 2016년 일별 역별 시간대별 승하차인원(1_8호선).csv',encoding='euc-kr')
#2017
subway_timelog3 = pd.read_excel('./일별시간별지하철승하차/서울교통공사 2017년 일별 역별 시간대별 승하차인원(1_8호선).xlsx')
subway_timelog3.columns = subway_timelog3.iloc[0,:].tolist()
subway_timelog3 = subway_timelog3.drop(index=0)
#2018
subway_timelog4 = pd.read_excel('./일별시간별지하철승하차/서울교통공사 2018년 일별 역별 시간대별 승하차인원(1_8호선).xlsx')
subway_timelog4.columns = subway_timelog4.iloc[0,:].tolist()
subway_timelog4 = subway_timelog4.drop(index=0)

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
subway_timelog33 = subway_timelog3.iloc[:,[0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]]
subway_timelog33.rename(columns={'00 ~ 01':'24~'},inplace=True)

In [39]:
subway_timelog44 = subway_timelog4.iloc[:,[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
subway_timelog44.rename(columns={'00 ~ 01':'24~'},inplace=True)

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
col_ls = []
for i in range(len(subway_timelog44.columns.tolist())):
    col_ls.append(re.sub(' ','',subway_timelog44.columns.tolist()[i]))

In [57]:
subway_timelog44.columns = col_ls

In [59]:
subway_daily_timelog = pd.concat([subway_timelog1,subway_timelog2,subway_timelog33,subway_timelog44])

In [90]:
subway_daily_timelog.rename(columns={'날짜':'USE_DT','역번호':'subway_station_code','역명':'subway_station_name','구분':'division'},inplace=True)

In [91]:
subway_daily_timelog.to_csv('seoul_subway_daily_timely_boarding_2015_2018.csv',index=False)

# seoul station english name crawling from wiki